### Alias算法
* Alias算法的作用：给定一个概率分布probs，$O(n)$时间建立一个查找table，通过查这个table可以在$O(1)$时间内产生服从该分布的一个样本；
* 第一步：alias_setup函数的作用是给定一个概率分布，产生两个数组（即查找表），`J`和`q`，如：
```python
J, q = alias_setup([0.1, 0.2 ,0.2, 0.5])
print(J,"\n", q)
[3 3 3 0] 
[0.4       0.8       0.8       0.9999999]
```
    * `J`数组的每一个位置都满足概率为1，其中索引`i`对应的值`J[i]`代表着索引`J[i]`处借了一部分概率给索引`i`，因为必须都为1;
    * `q`数组中索引`i`处的值代表该去掉从其他索引处借来概率后，本来表示`i`这个位置的概率。

* 第二步：alias_draw函数的作用是从该概率分布上取样一个样本，在$O(1)$时间内产生服从该分布的一个样本；

In [9]:
import numpy as np
from collections import deque

In [10]:
# Author：ZC
class Alias:
    def __init__(self, prolist):
        self.pro = prolist
        self.Initialization()
    def Initialization(self):
        self.N = len(self.pro)
        self.alias, self.prob = [0] * self.N, [0] * self.N
        small, large = deque(), deque()
        self.pro *= self.N
        for i in range(self.N):
            if self.pro[i] < 1:
                small.append(i)
            else:
                large.append(i)
        while small and large:
            l = small.popleft()
            g = large.popleft()
            self.prob[l] = self.pro[i]
            self.alias[l] = g
            self.pro[g] = self.pro[g] + self.pro[l] - 1
        if self.pro[g] < 1:
            small.append(g)
        else:
            large.append(g)
        while large:
            g = large.popleft()
            self.prob[g] = 1
        while small:
            l = small.popleft()
            self.prob[l] = 1
    def Generation(self):
        die = int(np.floor(np.random.random(1) * self.N))
        coin = np.random.binomial(1,self.prob[die])
        print("die Roll: %s coin Toss: %s" %(die,coin))
        if coin:
            return die
        else:
            return self.alias[die]


In [11]:
pro = np.array([1/2,1/3,1/12,1/12],dtype=np.float32)
pro # array([0.5       , 0.33333334, 0.08333334, 0.08333334], dtype=float32)
sampling = Alias(pro)

sampling.Generation()

die Roll: 2 coin Toss: 0


0

In [15]:

import numpy        as np
import numpy.random as npr

def alias_setup(probs):
    K = len(probs)
    q = np.zeros(K)
    J = np.zeros(K, dtype=np.int)
    # Sort the data into the outcomes with probabilities
    # that are larger and smaller than 1/K.
    smaller = []
    larger  = []
    for kk, prob in enumerate(probs):
        q[kk] = K*prob
        if q[kk] < 1.0:
            smaller.append(kk)
        else:
            larger.append(kk)

    # Loop though and create little binary mixtures that
    # appropriately allocate the larger outcomes over the
    # overall uniform mixture.
    while len(smaller) > 0 and len(larger) > 0:
        small = smaller.pop()
        large = larger.pop()

        J[small] = large
        q[large] = q[large] - (1.0 - q[small])

        if q[large] < 1.0:
            smaller.append(large)
        else:
            larger.append(large)

    return J, q

def alias_draw(J, q):
    K  = len(J)
    # Draw from the overall uniform mixture.
    kk = int(np.floor(npr.rand()*K))
    # Draw from the binary mixture, either keeping the
    # small one, or choosing the associated larger one.
    if npr.rand() < q[kk]:
        return kk
    else:
        return J[kk]

K = 5
N = 1000

# Get a random probability vector.
probs = npr.dirichlet(np.ones(K), 1).ravel()

# Construct the table.
J, q = alias_setup(probs)

# Generate variates.
X = np.zeros(N)
for nn in range(N):
    X[nn] = alias_draw(J, q)

In [13]:
alias_setup(pro)

(array([0, 0, 0, 0]), array([5.33333302, 2.66666698, 1.33333337, 1.33333337]))

In [17]:
def alias_setup(probs):
    '''
    Compute utility lists for non-uniform sampling from discrete distributions.
    Refer to https://hips.seas.harvard.edu/blog/2013/03/03/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
    for details
    '''
    K = len(probs)
    q = np.zeros(K, dtype=np.float32)
    J = np.zeros(K, dtype=np.int32)

    smaller = []
    larger = []
    for kk, prob in enumerate(probs):
        q[kk] = K*prob
        if q[kk] < 1.0:
            smaller.append(kk)
        else:
            larger.append(kk)

    while len(smaller) > 0 and len(larger) > 0:
        small = smaller.pop()
        large = larger.pop()

        J[small] = large
        q[large] = q[large] + q[small] - 1.0
        if q[large] < 1.0:
            smaller.append(large)
        else:
            larger.append(large)

    return J, q

In [216]:
J, q = alias_setup([0.2, 0.2 ,0.2, 0.2, 0.2])

In [217]:
print(J, q)

[0 0 0 0 0] [1. 1. 1. 1. 1.]


In [224]:
from collections import defaultdict
res = defaultdict(int)
for _ in range(10000):
    res[alias_draw(J,q)] += 1
res

defaultdict(int, {4: 2008, 0: 2025, 1: 1948, 3: 1977, 2: 2042})

In [45]:
def alias_draw(J, q):
    K  = len(J)

    # Draw from the overall uniform mixture.
    kk = int(np.floor(npr.rand()*K))

    # Draw from the binary mixture, either keeping the
    # small one, or choosing the associated larger one.
    if npr.rand() < q[kk]:
        return kk
    else:
        return J[kk]

In [49]:
K = 5
N = 10

# Get a random probability vector.
probs = npr.dirichlet(np.ones(K), 1).ravel()

# Construct the table.
J, q = alias_setup(probs)

# Generate variates.
X = np.zeros(N)
for nn in range(N):
    X[nn] = alias_draw(J, q)

In [50]:
X

array([1., 3., 1., 1., 2., 2., 1., 4., 3., 1.])

### Dataset 测试

In [23]:
np.random.RandomState?

In [24]:
import sys

In [26]:
sys.version_info > (3, 6)

True

In [27]:
path = "/Users/zhengchu/Documents/blogs/personal/RUGCN/data/"

> Dataset | Classes |  Features | Nodes | Edges

> CORA    | 7        | 1433     | 2,708  | 5,278 

In [30]:

import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import sys


def parse_index_file(filename):
    """Parse index file."""
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index


def sample_mask(idx, l):
    """Create mask."""
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)


def load_data(dataset_str):
    """
    Loads input data from gcn/data directory
    ind.dataset_str.x => the feature vectors of the training instances as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.tx => the feature vectors of the test instances as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.allx => the feature vectors of both labeled and unlabeled training instances
        (a superset of ind.dataset_str.x) as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.y => the one-hot labels of the labeled training instances as numpy.ndarray object;
    ind.dataset_str.ty => the one-hot labels of the test instances as numpy.ndarray object;
    ind.dataset_str.ally => the labels for instances in ind.dataset_str.allx as numpy.ndarray object;
    ind.dataset_str.graph => a dict in the format {index: [index_of_neighbor_nodes]} as collections.defaultdict
        object;
    ind.dataset_str.test.index => the indices of test instances in graph, for the inductive setting as list object.
    All objects above must be saved using python pickle module.
    :param dataset_str: Dataset name
    :return: All data input files loaded (as well the training/test data).
    """
    names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
    objects = []
    for i in range(len(names)):
        with open("/Users/zhengchu/Documents/blogs/personal/RUGCN/data/ind.{}.{}".format(dataset_str, names[i]), 'rb') as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.load(f, encoding='latin1'))
            else:
                objects.append(pkl.load(f))

    x, y, tx, ty, allx, ally, graph = tuple(objects)
    test_idx_reorder = parse_index_file("/Users/zhengchu/Documents/blogs/personal/RUGCN/data/ind.{}.test.index".format(dataset_str))
    test_idx_range = np.sort(test_idx_reorder)

    if dataset_str == 'citeseer':
        # Fix citeseer dataset (there are some isolated nodes in the graph)
        # Find isolated nodes, add them as zero-vecs into the right position
        test_idx_range_full = range(min(test_idx_reorder), max(test_idx_reorder)+1)
        tx_extended = sp.lil_matrix((len(test_idx_range_full), x.shape[1]))
        tx_extended[test_idx_range-min(test_idx_range), :] = tx
        tx = tx_extended
        ty_extended = np.zeros((len(test_idx_range_full), y.shape[1]))
        ty_extended[test_idx_range-min(test_idx_range), :] = ty
        ty = ty_extended

    features = sp.vstack((allx, tx)).tolil()
    features[test_idx_reorder, :] = features[test_idx_range, :]
    adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))

    labels = np.vstack((ally, ty))
    labels[test_idx_reorder, :] = labels[test_idx_range, :]

    idx_test = test_idx_range.tolist()
    idx_train = range(len(y))
    idx_val = range(len(y), len(y)+500)

    train_mask = sample_mask(idx_train, labels.shape[0])
    val_mask = sample_mask(idx_val, labels.shape[0])
    test_mask = sample_mask(idx_test, labels.shape[0])

    y_train = np.zeros(labels.shape)
    y_val = np.zeros(labels.shape)
    y_test = np.zeros(labels.shape)
    y_train[train_mask, :] = labels[train_mask, :]
    y_val[val_mask, :] = labels[val_mask, :]
    y_test[test_mask, :] = labels[test_mask, :]

    return adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask


def sparse_to_tuple(sparse_mx):
    """Convert sparse matrix to tuple representation."""
    def to_tuple(mx):
        if not sp.isspmatrix_coo(mx):
            mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose()
        values = mx.data
        shape = mx.shape
        return coords, values, shape

    if isinstance(sparse_mx, list):
        for i in range(len(sparse_mx)):
            sparse_mx[i] = to_tuple(sparse_mx[i])
    else:
        sparse_mx = to_tuple(sparse_mx)

    return sparse_mx


def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return sparse_to_tuple(features)


def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()


def preprocess_adj(adj):
    """Preprocessing of adjacency matrix for simple GCN model and conversion to tuple representation."""
    adj_normalized = normalize_adj(adj + sp.eye(adj.shape[0]))
    return sparse_to_tuple(adj_normalized)


def construct_feed_dict(features, support, labels, labels_mask, placeholders):
    """Construct feed dictionary."""
    feed_dict = dict()
    feed_dict.update({placeholders['labels']: labels})
    feed_dict.update({placeholders['labels_mask']: labels_mask})
    feed_dict.update({placeholders['features']: features})
    feed_dict.update({placeholders['support'][i]: support[i] for i in range(len(support))})
    feed_dict.update({placeholders['num_features_nonzero']: features[1].shape})
    return feed_dict


def chebyshev_polynomials(adj, k):
    """Calculate Chebyshev polynomials up to order k. Return a list of sparse matrices (tuple representation)."""
    print("Calculating Chebyshev polynomials up to order {}...".format(k))

    adj_normalized = normalize_adj(adj)
    laplacian = sp.eye(adj.shape[0]) - adj_normalized
    largest_eigval, _ = eigsh(laplacian, 1, which='LM')
    scaled_laplacian = (2. / largest_eigval[0]) * laplacian - sp.eye(adj.shape[0])

    t_k = list()
    t_k.append(sp.eye(adj.shape[0]))
    t_k.append(scaled_laplacian)

    def chebyshev_recurrence(t_k_minus_one, t_k_minus_two, scaled_lap):
        s_lap = sp.csr_matrix(scaled_lap, copy=True)
        return 2 * s_lap.dot(t_k_minus_one) - t_k_minus_two

    for i in range(2, k+1):
        t_k.append(chebyshev_recurrence(t_k[-1], t_k[-2], scaled_laplacian))

    return sparse_to_tuple(t_k)

In [69]:
dataset = 'citeseer'

In [66]:
names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
objects = []
for i in range(len(names)):
    with open("/Users/zhengchu/Documents/blogs/personal/RUGCN/data/ind.{}.{}".format(dataset, names[i]), 'rb') as f:
        if sys.version_info > (3, 0):
            objects.append(pkl.load(f, encoding='latin1'))
        else:
            objects.append(pkl.load(f))

x, y, tx, ty, allx, ally, graph = tuple(objects)

In [ ]:
    """
    Loads input data from gcn/data directory
    ind.dataset_str.x => the feature vectors of the training instances as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.tx => the feature vectors of the test instances as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.allx => the feature vectors of both labeled and unlabeled training instances
        (a superset of ind.dataset_str.x) as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.y => the one-hot labels of the labeled training instances as numpy.ndarray object;
    ind.dataset_str.ty => the one-hot labels of the test instances as numpy.ndarray object;
    ind.dataset_str.ally => the labels for instances in ind.dataset_str.allx as numpy.ndarray object;
    ind.dataset_str.graph => a dict in the format {index: [index_of_neighbor_nodes]} as collections.defaultdict
        object;
    ind.dataset_str.test.index => the indices of test instances in graph, for the inductive setting as list object.
    All objects above must be saved using python pickle module.
    :param dataset_str: Dataset name
    :return: All data input files loaded (as well the training/test data).
    """

In [70]:
for it in [x, y, tx, ty, allx, ally]:
    print(it.shape)

(140, 1433)
(140, 7)
(1000, 1433)
(1000, 7)
(1708, 1433)
(1708, 7)


In [74]:
list(graph.keys())[-2:]

[2706, 2707]

In [63]:
test_idx_reorder = parse_index_file("/Users/zhengchu/Documents/blogs/personal/RUGCN/data/ind.{}.test.index".format(dataset))
test_idx_range = np.sort(test_idx_reorder)

if dataset == 'citeseer':
    # Fix citeseer dataset (there are some isolated nodes in the graph)
    # Find isolated nodes, add them as zero-vecs into the right position
    test_idx_range_full = range(min(test_idx_reorder), max(test_idx_reorder)+1)
    tx_extended = sp.lil_matrix((len(test_idx_range_full), x.shape[1]))
    tx_extended[test_idx_range-min(test_idx_range), :] = tx
    tx = tx_extended
    ty_extended = np.zeros((len(test_idx_range_full), y.shape[1]))
    ty_extended[test_idx_range-min(test_idx_range), :] = ty
    ty = ty_extended

In [65]:
test_idx_range

array([1708, 1709, 1710, 1711, 1712, 1713, 1714, 1715, 1716, 1717, 1718,
       1719, 1720, 1721, 1722, 1723, 1724, 1725, 1726, 1727, 1728, 1729,
       1730, 1731, 1732, 1733, 1734, 1735, 1736, 1737, 1738, 1739, 1740,
       1741, 1742, 1743, 1744, 1745, 1746, 1747, 1748, 1749, 1750, 1751,
       1752, 1753, 1754, 1755, 1756, 1757, 1758, 1759, 1760, 1761, 1762,
       1763, 1764, 1765, 1766, 1767, 1768, 1769, 1770, 1771, 1772, 1773,
       1774, 1775, 1776, 1777, 1778, 1779, 1780, 1781, 1782, 1783, 1784,
       1785, 1786, 1787, 1788, 1789, 1790, 1791, 1792, 1793, 1794, 1795,
       1796, 1797, 1798, 1799, 1800, 1801, 1802, 1803, 1804, 1805, 1806,
       1807, 1808, 1809, 1810, 1811, 1812, 1813, 1814, 1815, 1816, 1817,
       1818, 1819, 1820, 1821, 1822, 1823, 1824, 1825, 1826, 1827, 1828,
       1829, 1830, 1831, 1832, 1833, 1834, 1835, 1836, 1837, 1838, 1839,
       1840, 1841, 1842, 1843, 1844, 1845, 1846, 1847, 1848, 1849, 1850,
       1851, 1852, 1853, 1854, 1855, 1856, 1857, 18

### Split数据测试

In [169]:
import numpy as np
import scipy.sparse as sp
from sklearn.model_selection import StratifiedKFold
import os
import pickle as pkl
import networkx as nx
import sys
import torch
from collections import defaultdict
from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer


def _eliminate_self_loops(A):
    """Remove self-loops from the adjacency matrix."""
    A = A.tolil()
    A.setdiag(0)
    A = A.tocsr()
    A.eliminate_zeros()
    return A

def eliminate_self_loops(G):
    G.adj_matrix = _eliminate_self_loops(G.adj_matrix)
    return G


def largest_connected_components(sparse_graph, n_components=1):
    """Select the largest connected components in the graph.

    Parameters
    ----------
    sparse_graph : SparseGraph
        Input graph.
    n_components : int, default 1
        Number of largest connected components to keep.

    Returns
    -------
    sparse_graph : SparseGraph
        Subgraph of the input graph where only the nodes in largest n_components are kept.

    """
    _, component_indices = sp.csgraph.connected_components(sparse_graph.adj_matrix)
    component_sizes = np.bincount(component_indices)
    components_to_keep = np.argsort(component_sizes)[::-1][:n_components]  # reverse order to sort descending
    nodes_to_keep = [
        idx for (idx, component) in enumerate(component_indices) if component in components_to_keep
    ]
    return create_subgraph(sparse_graph, nodes_to_keep=nodes_to_keep)

def get_train_val_test_split(random_state,
                             labels,
                             train_examples_per_class=None, val_examples_per_class=None,
                             test_examples_per_class=None,
                             train_size=None, val_size=None, test_size=None):
    num_samples, num_classes = labels.shape
    remaining_indices = list(range(num_samples))

    if train_examples_per_class is not None:
        train_indices = sample_per_class(random_state, labels, train_examples_per_class)
    else:
        # select train examples with no respect to class distribution
        train_indices = random_state.choice(remaining_indices, train_size, replace=False)

    if val_examples_per_class is not None:
        val_indices = sample_per_class(random_state, labels, val_examples_per_class, forbidden_indices=train_indices)
    else:
        remaining_indices = np.setdiff1d(remaining_indices, train_indices)
        val_indices = random_state.choice(remaining_indices, val_size, replace=False)

    forbidden_indices = np.concatenate((train_indices, val_indices))
    if test_examples_per_class is not None:
        test_indices = sample_per_class(random_state, labels, test_examples_per_class,
                                        forbidden_indices=forbidden_indices)
    elif test_size is not None:
        remaining_indices = np.setdiff1d(remaining_indices, forbidden_indices)
        test_indices = random_state.choice(remaining_indices, test_size, replace=False)
    else:
        test_indices = np.setdiff1d(remaining_indices, forbidden_indices)

    # assert that there are no duplicates in sets
    assert len(set(train_indices)) == len(train_indices)
    assert len(set(val_indices)) == len(val_indices)
    assert len(set(test_indices)) == len(test_indices)
    # assert sets are mutually exclusive
    assert len(set(train_indices) - set(val_indices)) == len(set(train_indices))
    assert len(set(train_indices) - set(test_indices)) == len(set(train_indices))
    assert len(set(val_indices) - set(test_indices)) == len(set(val_indices))
    if test_size is None and test_examples_per_class is None:
        # all indices must be part of the split
        assert len(np.concatenate((train_indices, val_indices, test_indices))) == num_samples

    if train_examples_per_class is not None:
        train_labels = labels[train_indices, :]
        train_sum = np.sum(train_labels, axis=0)
        # assert all classes have equal cardinality
        assert np.unique(train_sum).size == 1

    if val_examples_per_class is not None:
        val_labels = labels[val_indices, :]
        val_sum = np.sum(val_labels, axis=0)
        # assert all classes have equal cardinality
        assert np.unique(val_sum).size == 1

    if test_examples_per_class is not None:
        test_labels = labels[test_indices, :]
        test_sum = np.sum(test_labels, axis=0)
        # assert all classes have equal cardinality
        assert np.unique(test_sum).size == 1

    return train_indices, val_indices, test_indices


def sample_per_class(random_state, labels, num_examples_per_class, forbidden_indices=None):
    num_samples, num_classes = labels.shape
    sample_indices_per_class = {index: [] for index in range(num_classes)}

    # get indices sorted by class
    for class_index in range(num_classes):
        for sample_index in range(num_samples):
            if labels[sample_index, class_index] > 0.0:
                if forbidden_indices is None or sample_index not in forbidden_indices:
                    sample_indices_per_class[class_index].append(sample_index)

    # get specified number of indices for each class
    return np.concatenate(
        [random_state.choice(sample_indices_per_class[class_index], num_examples_per_class, replace=False)
         for class_index in range(len(sample_indices_per_class))
         ])

def is_binary_bag_of_words(features):
    features_coo = features.tocoo()
    return all(single_entry == 1.0 for _, _, single_entry in zip(features_coo.row, features_coo.col, features_coo.data))

def to_binary_bag_of_words(features):
    """Converts TF/IDF features to binary bag-of-words features."""
    features_copy = features.tocsr()
    features_copy.data[:] = 1.0
    return features_copy

def binarize_labels(labels, sparse_output=False, return_classes=False):
    """Convert labels vector to a binary label matrix.

    In the default single-label case, labels look like
    labels = [y1, y2, y3, ...].
    Also supports the multi-label format.
    In this case, labels should look something like
    labels = [[y11, y12], [y21, y22, y23], [y31], ...].

    Parameters
    ----------
    labels : array-like, shape [num_samples]
        Array of node labels in categorical single- or multi-label format.
    sparse_output : bool, default False
        Whether return the label_matrix in CSR format.
    return_classes : bool, default False
        Whether return the classes corresponding to the columns of the label matrix.

    Returns
    -------
    label_matrix : np.ndarray or sp.csr_matrix, shape [num_samples, num_classes]
        Binary matrix of class labels.
        num_classes = number of unique values in "labels" array.
        label_matrix[i, k] = 1 <=> node i belongs to class k.
    classes : np.array, shape [num_classes], optional
        Classes that correspond to each column of the label_matrix.

    """
    if hasattr(labels[0], '__iter__'):  # labels[0] is iterable <=> multilabel format
        binarizer = MultiLabelBinarizer(sparse_output=sparse_output)
    else:
        binarizer = LabelBinarizer(sparse_output=sparse_output)
    label_matrix = binarizer.fit_transform(labels).astype(np.float32)
    return (label_matrix, binarizer.classes_) if return_classes else label_matrix

In [164]:
class SparseGraph:
    """Attributed labeled graph stored in sparse matrix form.

    """
    def __init__(self, adj_matrix, attr_matrix=None, labels=None,
                 node_names=None, attr_names=None, class_names=None, metadata=None):
        """Create an attributed graph.

        Parameters
        ----------
        adj_matrix : sp.csr_matrix, shape [num_nodes, num_nodes]
            Adjacency matrix in CSR format.
        attr_matrix : sp.csr_matrix or np.ndarray, shape [num_nodes, num_attr], optional
            Attribute matrix in CSR or numpy format.
        labels : np.ndarray, shape [num_nodes], optional
            Array, where each entry represents respective node's label(s).
        node_names : np.ndarray, shape [num_nodes], optional
            Names of nodes (as strings).
        attr_names : np.ndarray, shape [num_attr]
            Names of the attributes (as strings).
        class_names : np.ndarray, shape [num_classes], optional
            Names of the class labels (as strings).
        metadata : object
            Additional metadata such as text.

        """
        # Make sure that the dimensions of matrices / arrays all agree
        if sp.isspmatrix(adj_matrix):
            adj_matrix = adj_matrix.tocsr().astype(np.float32)
        else:
            raise ValueError("Adjacency matrix must be in sparse format (got {0} instead)"
                             .format(type(adj_matrix)))

        if adj_matrix.shape[0] != adj_matrix.shape[1]:
            raise ValueError("Dimensions of the adjacency matrix don't agree")

        if attr_matrix is not None:
            if sp.isspmatrix(attr_matrix):
                attr_matrix = attr_matrix.tocsr().astype(np.float32)
            elif isinstance(attr_matrix, np.ndarray):
                attr_matrix = attr_matrix.astype(np.float32)
            else:
                raise ValueError("Attribute matrix must be a sp.spmatrix or a np.ndarray (got {0} instead)"
                                 .format(type(attr_matrix)))

            if attr_matrix.shape[0] != adj_matrix.shape[0]:
                raise ValueError("Dimensions of the adjacency and attribute matrices don't agree")

        if labels is not None:
            if labels.shape[0] != adj_matrix.shape[0]:
                raise ValueError("Dimensions of the adjacency matrix and the label vector don't agree")

        if node_names is not None:
            if len(node_names) != adj_matrix.shape[0]:
                raise ValueError("Dimensions of the adjacency matrix and the node names don't agree")

        if attr_names is not None:
            if len(attr_names) != attr_matrix.shape[1]:
                raise ValueError("Dimensions of the attribute matrix and the attribute names don't agree")

        self.adj_matrix = adj_matrix
        self.attr_matrix = attr_matrix
        self.labels = labels
        self.node_names = node_names
        self.attr_names = attr_names
        self.class_names = class_names
        self.metadata = metadata

    def num_nodes(self):
        """Get the number of nodes in the graph."""
        return self.adj_matrix.shape[0]

    def num_edges(self):
        """Get the number of edges in the graph.

        For undirected graphs, (i, j) and (j, i) are counted as single edge.
        """
        if self.is_directed():
            return int(self.adj_matrix.nnz)
        else:
            return int(self.adj_matrix.nnz / 2)

    def get_neighbors(self, idx):
        """Get the indices of neighbors of a given node.

        Parameters
        ----------
        idx : int
            Index of the node whose neighbors are of interest.

        """
        return self.adj_matrix[idx].indices

    def is_directed(self):
        """Check if the graph is directed (adjacency matrix is not symmetric)."""
        return (self.adj_matrix != self.adj_matrix.T).sum() != 0

    def to_undirected(self):
        """Convert to an undirected graph (make adjacency matrix symmetric)."""
        if self.is_weighted():
            raise ValueError("Convert to unweighted graph first.")
        else:
            self.adj_matrix = self.adj_matrix + self.adj_matrix.T
            self.adj_matrix[self.adj_matrix != 0] = 1
        return self

    def is_weighted(self):
        """Check if the graph is weighted (edge weights other than 1)."""
        return np.any(np.unique(self.adj_matrix[self.adj_matrix != 0].A1) != 1)

    def to_unweighted(self):
        """Convert to an unweighted graph (set all edge weights to 1)."""
        self.adj_matrix.data = np.ones_like(self.adj_matrix.data)
        return self

    # Quality of life (shortcuts)
    def standardize(self):
        """Select the LCC of the unweighted/undirected/no-self-loop graph.

        All changes are done inplace.

        """
        G = self.to_unweighted().to_undirected()
        G = eliminate_self_loops(G)
        G = largest_connected_components(G, 1)
        print("is Graph un-symmetric after standardize: ", G.is_directed())
        # G = G.to_unweighted().to_undirected()
        print("is Graph directed after standardize: ", G.is_directed())
        return G

    def unpack(self):
        """Return the (A, X, z) triplet."""
        return self.adj_matrix, self.attr_matrix, self.labels

In [165]:
def load_npz_to_sparse_graph(file_name):
    """Load a SparseGraph from a Numpy binary file.

    Parameters
    ----------
    file_name : str
        Name of the file to load.

    Returns
    -------
    sparse_graph : SparseGraph
        Graph in sparse matrix format.

    """
    file_name = "/Users/zhengchu/Documents/blogs/personal/RUGCN/data/{}".format(file_name)
    with np.load(file_name,  allow_pickle=True) as loader:
        loader = dict(loader)
        adj_matrix = sp.csr_matrix((loader['adj_data'], loader['adj_indices'], loader['adj_indptr']),
                                   shape=loader['adj_shape'])

        if 'attr_data' in loader:
            # Attributes are stored as a sparse CSR matrix
            attr_matrix = sp.csr_matrix((loader['attr_data'], loader['attr_indices'], loader['attr_indptr']),
                                        shape=loader['attr_shape'])
        elif 'attr_matrix' in loader:
            # Attributes are stored as a (dense) np.ndarray
            attr_matrix = loader['attr_matrix']
        else:
            attr_matrix = None

        if 'labels_data' in loader:
            # Labels are stored as a CSR matrix
            labels = sp.csr_matrix((loader['labels_data'], loader['labels_indices'], loader['labels_indptr']),
                                   shape=loader['labels_shape'])
        elif 'labels' in loader:
            # Labels are stored as a numpy array
            labels = loader['labels']
        else:
            labels = None

        node_names = loader.get('node_names')
        attr_names = loader.get('attr_names')
        class_names = loader.get('class_names')
        metadata = loader.get('metadata')

    return SparseGraph(adj_matrix, attr_matrix, labels, node_names, attr_names, class_names, metadata)

In [166]:

def create_subgraph(sparse_graph, _sentinel=None, nodes_to_remove=None, nodes_to_keep=None):
    """Create a graph with the specified subset of nodes.

    Exactly one of (nodes_to_remove, nodes_to_keep) should be provided, while the other stays None.
    Note that to avoid confusion, it is required to pass node indices as named arguments to this function.

    Parameters
    ----------
    sparse_graph : SparseGraph
        Input graph.
    _sentinel : None
        Internal, to prevent passing positional arguments. Do not use.
    nodes_to_remove : array-like of int
        Indices of nodes that have to removed.
    nodes_to_keep : array-like of int
        Indices of nodes that have to be kept.

    Returns
    -------
    sparse_graph : SparseGraph
        Graph with specified nodes removed.

    """
    # Check that arguments are passed correctly
    if _sentinel is not None:
        raise ValueError("Only call `create_subgraph` with named arguments',"
                         " (nodes_to_remove=...) or (nodes_to_keep=...)")
    if nodes_to_remove is None and nodes_to_keep is None:
        raise ValueError("Either nodes_to_remove or nodes_to_keep must be provided.")
    elif nodes_to_remove is not None and nodes_to_keep is not None:
        raise ValueError("Only one of nodes_to_remove or nodes_to_keep must be provided.")
    elif nodes_to_remove is not None:
        nodes_to_keep = [i for i in range(sparse_graph.num_nodes()) if i not in nodes_to_remove]
    elif nodes_to_keep is not None:
        nodes_to_keep = sorted(nodes_to_keep)
    else:
        raise RuntimeError("This should never happen.")

    sparse_graph.adj_matrix = sparse_graph.adj_matrix[nodes_to_keep][:, nodes_to_keep]
    if sparse_graph.attr_matrix is not None:
        sparse_graph.attr_matrix = sparse_graph.attr_matrix[nodes_to_keep]
    if sparse_graph.labels is not None:
        sparse_graph.labels = sparse_graph.labels[nodes_to_keep]
    if sparse_graph.node_names is not None:
        sparse_graph.node_names = sparse_graph.node_names[nodes_to_keep]
    return sparse_graph

In [167]:
dataset_graph = load_npz_to_sparse_graph(dataset + '.npz')

In [170]:
"""
1、先标准化图，即构造最大连通分量，去掉孤立节点
2、去掉自环
3、返回1、2步处理后得到的图的邻接矩阵adj，特征features，对应节点的标签labels

4、特征与标签进行特殊处理；
5、采样训练节点、验证节点和测试节点；
"""
dataset_graph = dataset_graph.standardize()
dataset_graph = eliminate_self_loops(dataset_graph)
adj, features, labels = dataset_graph.unpack()

labels = binarize_labels(labels)
if not is_binary_bag_of_words(features):
    print("Converting features of dataset {} to binary bag-of-words representation.".format(dataset_str[:-4]))
    features = to_binary_bag_of_words(features)
    
RandomState = np.random.RandomState(3)
idx_train, idx_val, idx_test = get_train_val_test_split(RandomState, labels, train_examples_per_class=20,
                                                        val_examples_per_class=30,
                                                        test_examples_per_class=None,
                                                        train_size=None, val_size=None, test_size=None)

is Graph un-symmetric after standardize:  False
is Graph directed after standardize:  False


In [147]:
print(f"adj shape: {adj.shape}, features shape: {features.shape}, labels shape: {labels.shape}")

adj shape: (2110, 2110), features shape: (2110, 3703), labels shape: (2110, 6)


In [148]:
print(f"train_size: {idx_train.size}, val_size: {idx_val.size}, test_size: {idx_test.size}")

train_size: 120, val_size: 180, test_size: 1810


#### adj是否对称

In [175]:
(adj != adj.T).sum()

0

In [176]:
raw_g, col_g = adj.nonzero()
index = list(zip(raw_g, col_g))

In [200]:
index[0][0]

0

In [204]:
adj[(1, 4)].astype(np.int32) == 0

True

In [199]:
cur_row = 4
_, cur_col = adj[cur_row,:].nonzero()
print(cur_row, cur_col)
print(np.ones(len(cur_col)).sum())

4 [ 488  712  734  982 1785 1862]
6.0


In [403]:
from tqdm import tqdm
class Walker:
    def __init__(self, adj, features, idx_train, idx_val, idx_test, num_walks, walk_length, p = 1.0, q = 1.0):
        self.adj = adj
        self.features = features
        self.idx_train = idx_train
        self.idx_val = idx_val
        self.idx_test = idx_test
        self.num_walks = num_walks
        self.walk_length = walk_length
        self.p = p
        self.q = q
        self.process_probs().simulate_walk(self.num_walks, walk_length)
    
    def simulate_walk(self, num_walks, walk_length):
        walks = defaultdict(list)
        for i in tqdm(range(num_walks)):
            print(f"Walk iteration: {i + 1}")
            index = np.concatenate([self.idx_val, self.idx_test])
            for idx in index:
                res = self.node2vec_walk(walk_length, idx)
                walks[idx] += res[1:]
        self.walks = walks
            
    def node2vec_walk(self, walk_length, start_node):
        walk = [start_node]
        while len(walk) < walk_length:
            cur = walk[-1]
            _, neighs = self.adj[cur, :].nonzero()
            if len(neighs) > 0:
                if len(walk) == 1:
                    res = neighs[
                            self.alias_draw(self.alias_nodes[cur][0]
                                            , self.alias_nodes[cur][1])]
                    if res in self.idx_train:
                        break
                    else:
                        walk.append(res)
                else:
                    prev = walk[-2]
                    edge = (prev, cur)
                    res = neighs[
                            self.alias_draw(self.alias_edges[edge][0],
                                            self.alias_edges[edge][1])]
                    if res in self.idx_train:
                        break
                    else:
                        walk.append(res)
            else:
                break
        return walk
        
    def process_probs(self):
        adj = self.adj
        alias_nodes = {}
        size = adj.shape[0]
        for cur_row in range(size):
            _, cur_col = adj[cur_row,:].nonzero()
            unnormalized_probs = np.ones(len(cur_col))
            deno = unnormalized_probs.sum()
            normalized_probs = unnormalized_probs / deno
            alias_nodes[cur_row] = self.alias_setup(normalized_probs)
        
        alias_edges = {}
        raw_g, col_g = adj.nonzero()
        for edge in zip(raw_g, col_g):
            alias_edges[edge] = self.get_alias_edges(edge[0], edge[1])
        self.alias_nodes = alias_nodes
        self.alias_edges = alias_edges
        return self
        
    def get_alias_edges(self, src, dst):
        unnormalized_probs = []
        _, neighs = self.adj[dst,:].nonzero()
        for neigh in neighs:
            if neigh == src:
                unnormalized_probs.append(1.0 / self.p)
            elif adj[(src, neigh)] or adj[(neigh,src)]:
                unnormalized_probs.append(1.0)
            else:
                unnormalized_probs.append(1.0 / self.q)
            deno = sum(unnormalized_probs)
            normalized_probs = [float(prob) / deno for prob in unnormalized_probs]
        return self.alias_setup(normalized_probs)
            
            
    def alias_setup(self, probs):
        '''
        Compute utility lists for non-uniform sampling from discrete distributions.
        Refer to https://hips.seas.harvard.edu/blog/2013/03/03/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
        for details
        '''
        K = len(probs)
        q = np.zeros(K, dtype=np.float32)
        J = np.zeros(K, dtype=np.int32)

        smaller = []
        larger = []
        for kk, prob in enumerate(probs):
            q[kk] = K*prob
            if q[kk] < 1.0:
                smaller.append(kk)
            else:
                larger.append(kk)

        while len(smaller) > 0 and len(larger) > 0:
            small = smaller.pop()
            large = larger.pop()

            J[small] = large
            q[large] = q[large] + q[small] - 1.0
            if q[large] < 1.0:
                smaller.append(large)
            else:
                larger.append(large)

        return J, q
    
    def alias_draw(self, J, q):
        K  = len(J)

        # Draw from the overall uniform mixture.
        kk = int(np.floor(npr.rand()*K))

        # Draw from the binary mixture, either keeping the
        # small one, or choosing the associated larger one.
        if npr.rand() < q[kk]:
            return kk
        else:
            return J[kk]


In [404]:
walker = Walker(adj, features, idx_train, idx_val, idx_test, num_walks=10, walk_length=10, p=1.0, q=1.0)


  0%|          | 0/10 [00:00<?, ?it/s]

Walk iteration: 1


 10%|█         | 1/10 [00:02<00:26,  2.90s/it]

Walk iteration: 2


 20%|██        | 2/10 [00:05<00:23,  2.91s/it]

...

 80%|████████  | 8/10 [00:22<00:05,  2.72s/it]

Walk iteration: 9


 90%|█████████ | 9/10 [00:25<00:02,  2.80s/it]

Walk iteration: 10


100%|██████████| 10/10 [00:27<00:00,  2.78s/it]


In [406]:
walks = walker.walks

In [408]:
unnormalized_probs = []
edges = []
for st_node, dest_nodes in walks.items():
    sigle_node_unnormalized_prob = []
    for dest_node, ct in Counter(dest_nodes).items():
        sigle_node_unnormalized_prob.append(ct)
        edges.append([st_node, dest_node])
    sigle_node_unnormalized_prob = np.array(sigle_node_unnormalized_prob)
    unnormalized_probs.append(sigle_node_unnormalized_prob / sigle_node_unnormalized_prob.sum())
unnormalized_probs = np.concatenate(unnormalized_probs)    

In [410]:
len(unnormalized_probs), len(edges)

(50200, 50200)

In [305]:
from collections import Counter

In [325]:
unnormalized_probs = []
edges = []
for dest_node in dest_nodes:
    sigle_node_unnormalized_prob = []
    for st_node, (k, v) in zip(st_nodes, Counter(dest_node).items()):
        sigle_node_unnormalized_prob.append(v)
        edges.append([st_node, k])
    sigle_node_unnormalized_prob = np.array(sigle_node_unnormalized_prob)
    sigle_node_unnormalized_prob = sigle_node_unnormalized_prob / sigle_node_unnormalized_prob.sum()
    unnormalized_probs.append(sigle_node_unnormalized_prob)

unnormalized_probs = np.concatenate(unnormalized_probs)

In [326]:
len(unnormalized_probs), len(edges)

(60682, 60682)

In [146]:
    def __dataset_generator(self, idx_train, idx_val, idx_test, neighbor):
        print("neighbor size is : ", neighbor)
        idx_train = idx_train.tolist()
        idx_val = idx_val.tolist()
        idx_test = idx_test.tolist()

        # 未标签节点与未标签节点间的边: 验证集节点
        uuvaledges = []
        for item in idx_val:
            neibors = list(self.G.neighbors(item))
            if len(neibors) != 0 and len(neibors) <= neighbor:
                for neibor in neibors:
                    if neibor not in idx_train:
                        uuvaledges.append([item, neibor])
        # 未标签节点与未标签节点间的边: 测试集节点
        uutestedges = []
        for item in idx_test:
            neibors = list(self.G.neighbors(item))
            if len(neibors) != 0 and len(neibors) <= neighbor:
                for neibor in neibors:
                    if neibor not in idx_train:
                        uutestedges.append([item, neibor])
        self.train_triples = torch.LongTensor(np.concatenate([uuvaledges, uutestedges]))
        self.index = self.train_triples.transpose(0, 1).contiguous()
        self.src = torch.as_tensor(list(set(self.index[0, :].tolist())))

        # self.train_triples = self.train_triples[torch.randperm(len(self.train_triples))]

        print("training edges size :", self.train_triples.size(), '\n')


In [ ]:
class 

In [140]:
for idx_tr in idx_train:
    print(adj[idx_tr].indices)

[ 546  994 1231 1246 1530]
[1016 1823]
[  14  247  618  740 1390 1910]
[665]
[ 156 1877]
[1156]
[ 400  409  574  593 1638 1639]
[   4  103 1324]
[ 227  563  625  641 1155 1953]
[1882]
[ 168  278 1204 1565 1566 1784]
[ 603  615 1656]
[2028 2029 2030]
[1626]
[ 273  855 1063 1731]
[ 14 175 247]
[ 368 1739]
[1252]
[1694]
[1015 1016 1705]
[1750]
[1348 1675 1687]
[ 423  757 1414 1438 1749]
[1791]
[1010]
[1348]
[475 860]
[ 471 1489]
[  24  401 1710 1969]
[ 790 1345]
[1586]
[ 570 1209 1415]
[  47 1437]
[1055 1492]
[ 481 1356 1437]
[1492 2108]
[1178]
[552]
[ 356 1251]
[1221]
[97]
[801]
[ 238  952 1575 2097 2099]
[  36  147  607 1152 1632]
[ 949 1923]
[  32   87  275  643 1088 1089]
[2094 2099]
[460]
[808 809 978]
[610]
[ 380  382  735  736  737  738 1158]
[1218 1778]
[ 121 1250 1511]
[211]
[1500]
[ 173 1171 1617 1618]
[871]
[  58 1978]
[ 169  389  400  964 1108 1154 1233 1404 1802]
[75]
[148 699]
[ 623 1498]
[ 794  846 1084 1868 1938]
[2003]
[  23 1451 1718 1839]
[ 182  977 1189 1691 1843]
[103

In [115]:
labels

array([[0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.]], dtype=float32)

In [117]:
RandomState = np.random.RandomState(3)
idx_train, idx_val, idx_test = get_train_val_test_split(RandomState, labels, train_examples_per_class=20,
                                                        val_examples_per_class=30,
                                                        test_examples_per_class=None,
                                                        train_size=None, val_size=None, test_size=None)

In [118]:
idx_train, idx_val, idx_test

(array([1531,   77,  106,  661, 1802, 1853,  654,  488, 1954, 1883, 1564,
        1782, 2027, 1719,  433, 1998,   22,  224, 1962,  822,  695,  346,
         649, 1792,  789,  111, 2047, 1587, 2044, 1488, 1585,  859,  571,
        1490, 1870,  560, 1116, 1660, 1041,  283, 1736,  596, 2094, 1631,
        1500,   88,  238, 1050,  807, 1834, 1341, 1779, 1262,  905, 1923,
        1619, 2074, 1628,  156,  386,  405,  196, 1908,  492,  843,  723,
        1115, 1214, 1766,  623, 1114, 1276, 1268, 1649,  271,  265,  990,
        1409, 1737, 1596, 1310,  604, 1366,  368, 1992,  400,  573,  329,
         950, 1090, 1578, 2078, 1603, 1252, 1440,  181, 1452,  899,  717,
         667, 1637, 1325, 1725, 1444,  652, 1007,  192,  605,  630,  640,
         436,  171,  882,  628, 1871,  784,  650, 1604,  636, 1971]),
 array([1332, 1600,   19,   10,  514,  919, 1285,  698,  925,  278,  353,
        1565,  627,  610, 1773,  726,  229,  612, 1566,  227, 1156,  616,
         318, 1064, 1982, 1713,  140, 1403

In [ ]:
adj.tocoo(), features.tocsr(), label.astype(np.int32)

In [93]:
        print("Randomly split dataset...")
        dataset_str += '.npz'
        dataset_graph = load_npz_to_sparse_graph(dataset_str)

        if standardize:
            dataset_graph = dataset_graph.standardize()
        else:
            # dataset_graph = dataset_graph.to_undirected()
            dataset_graph = eliminate_self_loops(dataset_graph)


        adj, features, labels = dataset_graph.unpack()
        labels = binarize_labels(labels)
        if not is_binary_bag_of_words(features):
            print("Converting features of dataset {} to binary bag-of-words representation.".format(dataset_str[:-4]))
            features = to_binary_bag_of_words(features)

        # adj matrix needs to be symmetric
        if standardize:
            assert (adj != adj.T).nnz == 0
        # features need to be binary bag-of-word vectors
        assert is_binary_bag_of_words(features), f"Non-binary node_features entry!"

        label = np.where(labels)[1]
        idx_train, idx_val, idx_test = get_train_val_test_split(RandomState, labels, train_examples_per_class=20,
                                                                val_examples_per_class=30,
                                                                test_examples_per_class=None,
                                                                train_size=None, val_size=None, test_size=None)

        return adj.tocoo(), features.tocsr(), label.astype(np.int32), torch.LongTensor(idx_train), torch.LongTensor(idx_val), torch.LongTensor(idx_test)

<3312x3312 sparse matrix of type '<class 'numpy.float32'>'
	with 4715 stored elements in Compressed Sparse Row format>

#### normalized KL 

In [331]:
import torch

In [334]:
probs = torch.FloatTensor([0.5, 0.4, 0.1])
kl = torch.FloatTensor([1, 2, 4])

In [335]:
probs * kl

tensor([0.5000, 0.8000, 0.4000])